In [630]:
import cv2
import numpy as np

In [ ]:
def detect_board(vid_frame_gray, ref_frame_gray, orb, bf):
    kp_board, des_board = orb.detectAndCompute(ref_frame_gray, None)
    kp_frame, des_frame = orb.detectAndCompute(vid_frame_gray, None)

    matches = bf.knnMatch(des_board, des_frame, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    if len(good) > 20:
        src_pts = np.float32(
            [kp_board[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32(
            [kp_frame[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    else:
        return None
    h, w = ref_frame_gray.shape[:2]

    H_inv = np.linalg.inv(H)
    board_corners = np.float32(
        [[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
    camera_corners = cv2.perspectiveTransform(board_corners, H_inv)
    return camera_corners


def detect_cards(frame_gray,):
    frame_gray = cv2.GaussianBlur(frame_gray, (5, 5), 0)
    frame_gray = cv2.adaptiveThreshold(
        frame_gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        31, 7)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    frame_gray = cv2.morphologyEx(frame_gray, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(
        frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    test = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(test, contours, contourIdx=-1,
                     color=(0, 0, 255), thickness=3, lineType=cv2.LINE_AA)
    cv2.imshow("Contours", test)

    cards = []

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 5000 or area > 100000:
            continue

        # Use minAreaRect for tilted boxes
        rect = cv2.minAreaRect(cnt)
        cards.append(rect)
    return cards


def sort_box_corners(box):
    s = box.sum(axis=1)
    d = np.diff(box, axis=1)

    tl = box[np.argmin(s)]
    br = box[np.argmax(s)]
    tr = box[np.argmin(d)]
    bl = box[np.argmax(d)]
    return tl, br, tr, bl


def calc_box_dimensions(tl, br, tr, bl):
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    return maxWidth, maxHeight


def extract_rotated_card(box, vid_frame):
    tl, br, tr, bl = sort_box_corners(box)
    box_width, box_height = calc_box_dimensions(tl, br, tr, bl)
    # Define source points (ordered) and destination points (standard flat rect)
    src_pts = np.array([tl, tr, br, bl], dtype="float32")

    dst_pts = np.array([
        [0, 0],
        [box_width - 1, 0],
        [box_width - 1, box_height - 1],
        [0, box_height - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(vid_frame, M, (box_width, box_height))

    if box_width > box_height:
        warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)
    return warped


def classify_card(extracted_rect):
    hsv = cv2.cvtColor(extracted_rect, cv2.COLOR_BGR2HSV)

    h, w, _ = extracted_rect.shape
    center_region = hsv[h//4:3*h//4, w//4:3*w//4]

    avg_color = np.mean(center_region, axis=(0, 1))
    H, S, V = avg_color[0], avg_color[1], avg_color[2]

    if V < 110:
        return "Black"

    if S < 40:
        return "White"

    if H < 16:
        return "Red"

    elif H < 24:
        return "Orange"

    elif H < 38:
        if S < 90:
            return "Quest"
        else:
            return "Yellow"

    elif H < 65:
        return "Green"

    elif H < 95:
        if S < 140:
            return "Joker"
        else:
            return "Blue"

    else:
        return "Pink"

In [ ]:
REFERENCE_CARDS = {
    "red":    (13.0, 179.3, 167.1),
    "orange": (18.8, 179.3, 187.8),
    "yellow": (30.8, 123.5, 192.0),
    "green":  (42.5, 153.3, 167.2),
    "blue":   (85.8, 188.0, 167.7),
    "pink":   (125.8, 58.0, 190.4),
    "black":  (37.0, 50.9,  78.7),
    "white":  (28.2, 33.1,  193.6),
    "joker":  (77.9, 94.8,  155.8),
    "quest":  (26.6, 52.7,  206.7)
}


def get_hsv_avg(card_img):
    hsv = cv2.cvtColor(card_img, cv2.COLOR_BGR2HSV)
    h, w, _ = card_img.shape
    center = hsv[h//10:9*h//10, w//10:9*w//10]
    avg = np.mean(center, axis=(0, 1))
    return avg


def classify_nearest_neighbor(card_img):
    input_hsv = get_hsv_avg(card_img)

    min_dist = float('inf')
    best_match = "Unknown"

    for name, ref_hsv in REFERENCE_CARDS.items():
        dh = abs(input_hsv[0] - ref_hsv[0])
        ds = abs(input_hsv[1] - ref_hsv[1])
        dv = abs(input_hsv[2] - ref_hsv[2])

        dist = (dh * 4)**2 + (ds * 1)**2 + (dv * 0.25)**2

        if dist < min_dist:
            min_dist = dist
            best_match = name

    return best_match

In [633]:
board = cv2.imread("../data/board_reference.jpg")
board_gray = cv2.cvtColor(board, cv2.COLOR_BGR2GRAY)

BOARD_H, BOARD_W = board.shape[:2]

cap = cv2.VideoCapture("../data/easy2_mid.mp4")
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)
ret, frame = cap.read()
TABLE_H, TABLE_W = frame.shape[:2]


# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [634]:
orb = orb = cv2.ORB.create(
    nfeatures=500,
    scaleFactor=1.15,
    nlevels=10,
    fastThreshold=7
)
bf = cv2.BFMatcher(cv2.NORM_HAMMING)

In [ ]:
cv2.namedWindow("Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Tracking", 900, 600)
# cv2.namedWindow("Board View", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Board View", 900,600)

cv2.namedWindow("Contours", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Contours", 900, 600)


i = 0
try:
    while True:
        ret, frame = cap.read()
        overlay = frame.copy()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if i % 30 == 0:
            board_corners_tmp = detect_board(frame_gray, board, orb, bf)

        if board_corners_tmp is not None:
            board_corners = board_corners_tmp

        pts = board_corners.astype(int)
        cv2.polylines(overlay, [pts], isClosed=True,
                      color=(0, 0, 255), thickness=3)
        H, _ = cv2.findHomography(board_corners, np.float32(
            [[0, 0], [BOARD_W, 0], [BOARD_W, BOARD_H], [0, BOARD_H]]))
        board_view = cv2.warpPerspective(frame, H, (BOARD_W, BOARD_H))

        # table_view = cv2.warpPerspective(frame, H_table, (TABLE_W, TABLE_H))

        # table_view = cv2.warpPerspective(frame, H, (TABLE_W, TABLE_H))

        detected_cards = detect_cards(frame_gray)
        for rect in detected_cards:
            box = cv2.boxPoints(rect)
            box = np.int64(box)

            cv2.drawContours(overlay, [box], 0, (0, 255, 0), 2)

            width = int(rect[1][0])
            height = int(rect[1][1])

            src_pts = box.astype("float32")
            # coordinate of the points in box points after the rectangle has been straightened
            dst_pts = np.array([[0, height-1],
                                [0, 0],
                                [width-1, 0],
                                [width-1, height-1]], dtype="float32")

            M = cv2.getPerspectiveTransform(src_pts, dst_pts)
            warped = cv2.warpPerspective(frame, M, (width, height))
            # cv2.imshow(f"{i}rect", warped)

        i = (i+1) % 1000
        cv2.imshow("Tracking", overlay)
        # cv2.imshow("Board View", board_view)

        if cv2.waitKey(25) & 0xFF == 27:
            break
except Exception as e:
    cap.release()
    cv2.destroyAllWindows()
    raise e

# cap.release()
cv2.destroyAllWindows()

In [ ]:
import time


cv2.namedWindow("Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Tracking", 900, 600)
# cv2.namedWindow("Board View", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Board View", 900,600)

cv2.namedWindow("Contours", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Contours", 900, 600)
cv2.namedWindow("Warped Card", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Warped Card", 900, 600)


ret, frame = cap.read()
overlay = frame.copy()
frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
if i % 30 == 0:
    board_corners_tmp = detect_board(frame_gray, board, orb, bf)

if board_corners_tmp is not None:
    board_corners = board_corners_tmp

pts = board_corners.astype(int)
cv2.polylines(overlay, [pts], isClosed=True,
              color=(0, 255, 0), thickness=3)
H, _ = cv2.findHomography(board_corners, np.float32(
    [[0, 0], [BOARD_W, 0], [BOARD_W, BOARD_H], [0, BOARD_H]]))
board_view = cv2.warpPerspective(frame, H, (BOARD_W, BOARD_H))

# table_view = cv2.warpPerspective(frame, H_table, (TABLE_W, TABLE_H))

# table_view = cv2.warpPerspective(frame, H, (TABLE_W, TABLE_H))


detected_cards = detect_cards(frame_gray)
extracted_rects = []
for i, rect in enumerate(detected_cards):
    box = cv2.boxPoints(rect)
    box = np.int64(box)

    cv2.drawContours(overlay, [box], 0, (0, 255, 0), 2)

    extracted_rect = extract_rotated_card(box, frame)
    card_type_nn = classify_nearest_neighbor(extracted_rect)
    card_type_elif = classify_card(extracted_rect)

    cv2.putText(overlay, card_type_nn,
                box[0], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(overlay, card_type_elif,
                box[1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    # extracted_rects.append(extracted_rect)


i = (i+1) % 1000
cv2.imshow("Tracking", overlay)
# cv2.imshow("Board View", board_view)
cv2.waitKey(0)
cv2.destroyAllWindows()